<a href="https://colab.research.google.com/github/npselvan/StockInventory/blob/stockphase1/StockTrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7&scripcode=500354&flag=sp&Submit=G

In [2]:
headers={
    'User-Agent':'Mozilla/5.0'
}
url="https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7&scripcode=500354&flag=sp&Submit=G"
data=requests.get(url, headers=headers).content
df_list=pd.read_html(data)
df=df_list[-1]
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
bsestocks=pd.read_csv('/content/drive/MyDrive/BSE_StockList.csv')
bsestocks.drop(columns=['Issuer Name'], axis=1, inplace=True)
bsestocks.head()

,Security Code,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500002,ABB,ABB India Limited,Active,A,2.00,INE117A01022,Heavy Electrical Equipment,Equity
1,500003,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.00,INE208C01025,Oil Marketing & Distribution,Equity
2,500008,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.00,INE885A01032,Auto Parts & Equipment,Equity
3,500009,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,X,10.00,INE432A01017,Pharmaceuticals,Equity
4,500010,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,2.00,INE001A01036,Housing Finance,Equity


In [51]:
activeStocks=bsestocks[bsestocks['Group']=='A ']
activeStocks.head()
df1=activeStocks.head(500)

In [52]:
df2=pd.DataFrame()
for idx, sc in enumerate(df1['Security Code']):
  url = "https://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7&scripcode={}&flag=sp&Submit=G".format(sc)
  pcontent = requests.get(url, headers=headers).content
  df=pd.read_html(pcontent)[-1]
  df['StockCode']=sc
  df2=df2.append(df,ignore_index=False) 
  

In [53]:
df2.drop(df2[df2[1]=='Open'].index, inplace=True)

In [54]:
df3=df2.copy()

In [55]:
cols=['Date','Open','High','Low','Close','WAP','No. of Shares','No. of Trades','Total Turnover','Deliverable Quantity','% Deli. Qty to Traded Qty','H-L','C-O','StockCode']
df3.columns=cols

In [56]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 2 to 31
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Date                       15000 non-null  object
 1   Open                       15000 non-null  object
 2   High                       15000 non-null  object
 3   Low                        15000 non-null  object
 4   Close                      15000 non-null  object
 5   WAP                        15000 non-null  object
 6   No. of Shares              15000 non-null  object
 7   No. of Trades              15000 non-null  object
 8   Total Turnover             15000 non-null  object
 9   Deliverable Quantity       15000 non-null  object
 10  % Deli. Qty to Traded Qty  15000 non-null  object
 11  H-L                        15000 non-null  object
 12  C-O                        15000 non-null  object
 13  StockCode                  15000 non-null  int64 
dtypes: int64(

In [57]:
df3 = df3.astype({'Open':'float32', 'High':'float32', 'Low':'float32', 'Close':'float32', 'WAP':'float32', 'No. of Shares':'int64',
            'No. of Trades':'int64','Total Turnover':'int64','Deliverable Quantity':'int64','% Deli. Qty to Traded Qty':'float32',
            'H-L':'float32','C-O':'float32'})

In [58]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 2 to 31
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       15000 non-null  object 
 1   Open                       15000 non-null  float32
 2   High                       15000 non-null  float32
 3   Low                        15000 non-null  float32
 4   Close                      15000 non-null  float32
 5   WAP                        15000 non-null  float32
 6   No. of Shares              15000 non-null  int64  
 7   No. of Trades              15000 non-null  int64  
 8   Total Turnover             15000 non-null  int64  
 9   Deliverable Quantity       15000 non-null  int64  
 10  % Deli. Qty to Traded Qty  15000 non-null  float32
 11  H-L                        15000 non-null  float32
 12  C-O                        15000 non-null  float32
 13  StockCode                  15000 non-null  int64 

In [59]:
df3.sort_values(['StockCode','C-O'],ascending=False).groupby('StockCode').head(3)

,Date,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,H-L,C-O,StockCode
28,26/05/21,11425.000000,11974.150391,11412.000000,11907.750000,11701.559570,28067,5186,328427814,4011,14.290000,562.150024,482.750000,532978
10,21/06/21,11721.000000,12202.950195,11721.000000,12161.700195,12044.190430,14422,3269,173701294,3295,22.850000,481.950012,440.700012,532978
21,4/06/21,11775.000000,12189.000000,11775.000000,12152.400391,12006.459961,20375,4294,244631580,6706,32.910000,414.000000,377.399994,532978
11,18/06/21,4046.550049,4192.000000,4030.399902,4164.950195,4114.140137,30211,4612,124292353,9385,31.059999,161.600006,118.400002,532977
2,1/07/21,4140.000000,4250.000000,4140.000000,4205.600098,4216.529785,26602,3351,112168134,2850,10.710000,110.000000,65.599998,532977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,31/05/21,339.000000,360.500000,339.000000,358.649994,355.170013,101482,3271,36043854,61242,60.349998,21.500000,19.650000,500003
30,24/05/21,332.450012,355.000000,332.450012,348.899994,349.029999,122089,5072,42612315,47072,38.560001,22.549999,16.450001,500003
25,31/05/21,1599.000000,1730.000000,1599.000000,1667.900024,1677.130005,75026,6213,125828418,22519,30.010000,131.000000,68.900002,500002
27,27/05/21,1518.000000,1600.000000,1493.849976,1576.500000,1547.430054,28890,2757,44705265,17745,61.419998,106.150002,58.500000,500002


In [50]:
activeStocks[activeStocks['Security Code']==500148]

,Security Code,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
72,500148,UFLEX,UFLEX LTD.,Active,A,10.00,INE516A01017,Containers & Packaging,Equity
